In [6]:
# Option1: open-source model
import torch
import spacy
from myselfcheckgpt.modeling_selfcheck import SelfCheckLLMPrompt


/home/srongaa/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
question = 'What did Petra van Staveren win a gold medal for?'
answer = 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.'
nlp = spacy.load("en_core_web_sm")
doc = nlp(answer)

In [3]:
#过滤掉stop word
token_not_stop = list(token.text for token in doc if not token.is_stop)
#without stop word
to_generates = []
for token_to_strip in token_not_stop:
    to_generate = []
    for token in doc:
        if token.text == token_to_strip:
            continue
        to_generate.append(token.text)
    to_generates.append(" ".join(to_generate))
print(answer)
to_generates

Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.


['van Stoveren won silver medal in the 2008 Summer Olympics in Beijing , China .',
 'Petra Stoveren won silver medal in the 2008 Summer Olympics in Beijing , China .',
 'Petra van won silver medal in the 2008 Summer Olympics in Beijing , China .',
 'Petra van Stoveren silver medal in the 2008 Summer Olympics in Beijing , China .',
 'Petra van Stoveren won medal in the 2008 Summer Olympics in Beijing , China .',
 'Petra van Stoveren won silver in the 2008 Summer Olympics in Beijing , China .',
 'Petra van Stoveren won silver medal in the Summer Olympics in Beijing , China .',
 'Petra van Stoveren won silver medal in the 2008 Olympics in Beijing , China .',
 'Petra van Stoveren won silver medal in the 2008 Summer in Beijing , China .',
 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in , China .',
 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing China .',
 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing , .',
 'Pet

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
llm_model = "mistralai/Mistral-7B-Instruct-v0.2"
selfcheck_prompt = SelfCheckLLMPrompt(llm_model, device)

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00, 10.39it/s]


SelfCheck-LLMPrompt (mistralai/Mistral-7B-Instruct-v0.2) initialized to device cpu


In [11]:

# Option2: API access 
# (currently only support OpenAI and Groq)
# from selfcheckgpt.modeling_selfcheck_apiprompt import SelfCheckAPIPrompt
# selfcheck_prompt = SelfCheckAPIPrompt(client_type="openai", model="gpt-3.5-turbo")
# selfcheck_prompt = SelfCheckAPIPrompt(client_type="groq", model="llama3-70b-8192", api_key="your-api-key")
question = 'What did Petra van Staveren win a gold medal for?'
answer = 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.'

#to_split = answer
#sentence level splice

sample1 = question + "Petra van Staveren won two gold medals at the 1987 Summer Olympic games in Seoul, South Korea - one in women's 100m sprint and one in women's 100m freestyle"
sample2 = question + "Petra van Staveren won a gold medal for women's individual vault during the 2007 European Championship Gymnastics in Moscow, Russia?."
sample3 = question + "Petra van Staveren won a gold medal in judo at the 2017 Summer Olympic Games in South Korea."
#to_eval: strip a word.
for to_eval in to_generate:
    doc = nlp(to_eval)
    sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
    print(sentences)
    sent_scores_prompt = selfcheck_prompt.predict(
        sentences = sentences,                          # list of sentences
        sampled_passages = [sample1, sample2, sample3], # list of sampled passages
        verbose = True, # whether to show a progress bar
    )
    print(sent_scores_prompt)
# [0.33333333, 0.66666667] -- based on the example above

['Petra']


  0%|          | 0/1 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
from myselfcheckgpt/selfcheckgpt.modeling_selfcheck import SelfCheckMQAG
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_mqag = SelfCheckMQAG(device=device) # set device to 'cuda' if GPU is available

SelfCheck-MQAG initialized to device cpu


In [19]:
question = 'What did Petra van Staveren win a gold medal for?'
answer = 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.'

sample1 = question + "Petra van Staveren won two gold medals at the 1987 Summer Olympic games in Seoul, South Korea - one in women's 100m sprint and one in women's 100m freestyle"
sample2 = question + "Petra van Staveren won a gold medal for women's individual vault during the 2007 European Championship Gymnastics in Moscow, Russia?."
sample3 = question + "Petra van Staveren won a gold medal in judo at the 2017 Summer Olympic Games in South Korea."

passage = question + answer

#to_eval: strip a word.
for to_eval in to_generates:
    doc = nlp(to_eval)
    sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
    print(sentences)
    sent_scores_mqag = selfcheck_mqag.predict(
        sentences = sentences,               # list of sentences
        passage = passage,                   # passage (before sentence-split)
        sampled_passages = [sample1, sample2, sample3], # list of sampled passages
        num_questions_per_sent = 5,          # number of questions to be drawn  
        scoring_method = 'bayes_with_alpha', # options = 'counting', 'bayes', 'bayes_with_alpha'
        beta1 = 0.8, beta2 = 0.8,            # additional params depending on scoring_method
    )
    print(sent_scores_mqag)

['van Stoveren won silver medal in the 2008 Summer Olympics in Beijing , China .']


KeyboardInterrupt: 

In [5]:
from myselfcheckgpt.modeling_selfcheck import SelfCheckBERTScore, SelfCheckNgram
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_bertscore = SelfCheckBERTScore(rescale_with_baseline=True)
selfcheck_ngram = SelfCheckNgram(n=1) # n=1 means Unigram, n=2 means Bigram, etc.

SelfCheck-BERTScore initialized
SelfCheck-1gram initialized


In [6]:
import myselfcheckgpt
import importlib
importlib.reload(myselfcheckgpt)

<module 'myselfcheckgpt' from '/home/srongaa/llm/selfcheckgpt/myselfcheckgpt/__init__.py'>

In [1]:
question = 'What did Petra van Staveren win a gold medal for?'
answer = 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.'

sample1 = question + "Petra van Staveren won two gold medals at the 1987 Summer Olympic games in Seoul, South Korea - one in women's 100m sprint and one in women's 100m freestyle"
sample2 = question + "Petra van Staveren won a gold medal for women's individual vault during the 2007 European Championship Gymnastics in Moscow, Russia?."
sample3 = question + "Petra van Staveren won a gold medal in judo at the 2017 Summer Olympic Games in South Korea."

passage = question + answer

#to_eval: strip a word.
for to_eval in to_generates:
    doc = nlp(to_eval)
    sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
    print(sentences)
    sent_scores_ngram = selfcheck_ngram.predict(
        sentences = sentences,   
        passage = passage,
        sampled_passages = [sample1, sample2, sample3],
    )
    print(sent_scores_ngram)

NameError: name 'to_generates' is not defined

In [5]:

question = 'What did Petra van Staveren win a gold medal for?'
answer = 'Petra van Stoveren won silver medal in the 2008 Summer Olympics in Beijing, China.'

sample1 = question + "Petra van Staveren won two gold medals at the 1987 Summer Olympic games in Seoul, South Korea - one in women's 100m sprint and one in women's 100m freestyle"
sample2 = question + "Petra van Staveren won a gold medal for women's individual vault during the 2007 European Championship Gymnastics in Moscow, Russia?."
sample3 = question + "Petra van Staveren won a gold medal in judo at the 2017 Summer Olympic Games in South Korea."

passage = question + answer

#to_eval: strip a word.
to_eval = answer
doc = nlp(to_eval)
sentences = [sent.text.strip() for sent in doc.sents] # spacy sentence tokenization
print(sentences)
sent_scores_ngram = selfcheck_ngram.predict(
    sentences = sentences,   
    passage = passage,
    sampled_passages = [sample1, sample2, sample3],
)
print(sent_scores_ngram)

NameError: name 'nlp' is not defined

In [8]:
import numpy as np
lst_neg_logprob = [np.float64(-3.2386784521643803), np.float64(-3.0155349008501706), np.float64(-4.624972813284271), np.float64(-3.2386784521643803), np.float64(-4.624972813284271), np.float64(-3.2386784521643803), np.float64(-2.5455312716044354), np.float64(-3.2386784521643803), np.float64(-4.624972813284271), np.float64(-3.5263605246161616), np.float64(-4.624972813284271), np.float64(-2.5455312716044354), np.float64(-4.624972813284271), np.float64(-3.5263605246161616), np.float64(-4.624972813284271), np.float64(-3.5263605246161616)]
for i, token in enumerate(doc):
    print(token, lst_neg_logprob[i])


Petra -3.2386784521643803
van -3.0155349008501706
Stoveren -4.624972813284271
won -3.2386784521643803
silver -4.624972813284271
medal -3.2386784521643803
in -2.5455312716044354
the -3.2386784521643803
2008 -4.624972813284271
Summer -3.5263605246161616
Olympics -4.624972813284271
in -2.5455312716044354
Beijing -4.624972813284271
, -3.5263605246161616
China -4.624972813284271
. -3.5263605246161616


In [9]:
Stoveren -4.624972813284271
silver -4.624972813284271
2008 -4.624972813284271
Olympics -4.624972813284271
Beijing -4.624972813284271
China -4.624972813284271


NameError: name 'Stoveren' is not defined

In [ ]:
['St',
 'o',
 'veren',
 'silver',
 ' medal',
 '2008',
 ' Summer Olympics',
 ' in ',
 'Beijing, China']

['silver', '2008', 'Beijing, China']
